In [3]:
from local_pc.Test_basemodel_functions import  *
from forecasting_functions import *
from os import *

# loading the files
path_to_folder = "D:\AI_time_series_repos\TS_code\Forecasting\LSTM\Hypersearch\local_stateless2\outputs"
number = 2

makedirs(path.join(path_to_folder,"pictures"),exist_ok=True)
output_path_pictures = path.join(path_to_folder,"pictures")

error_0x1e84_name = "error_model"+str(number)+"_sl_0x1e84e4d5cf1f463147f3e4d566167597423d7769.csv"
error_0x78a_name = "error_model"+str(number)+"_sl_0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e.csv"
error_0xc3b_name = "error_model"+str(number)+"_sl_0xc3b2f61a72e188cfd44483fce1bc11d6a628766d.csv"

error_0x1e84 = pd.read_csv(path.join(path_to_folder,error_0x1e84_name),index_col=0)
error_0x78a = pd.read_csv(path.join(path_to_folder,error_0x78a_name),index_col=0)
error_0xc3b = pd.read_csv(path.join(path_to_folder,error_0xc3b_name),index_col=0)

logbook_0x1e84_name = "logbook_model"+str(number)+"_sl_0x1e84e4d5cf1f463147f3e4d566167597423d7769.csv"
logbook_0x78a_name = "logbook_model"+str(number)+"_sl_0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e.csv"
logbook_0xc3b_name = "logbook_model"+str(number)+"_sl_0xc3b2f61a72e188cfd44483fce1bc11d6a628766d.csv"

logbook_0x1e84 = pd.read_csv(path.join(path_to_folder,logbook_0x1e84_name),index_col=0)
logbook_0x78a = pd.read_csv(path.join(path_to_folder,logbook_0x78a_name),index_col=0)
logbook_0xc3b = pd.read_csv(path.join(path_to_folder,logbook_0xc3b_name),index_col=0)

training_result_0x1e84_name = "training_result_model"+str(number)+"_sl_0x1e84e4d5cf1f463147f3e4d566167597423d7769.csv"
training_result_0x78a_name = "training_result_model"+str(number)+"_sl_0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e.csv"
training_result_0xc3b_name = "training_result_model"+str(number)+"_sl_0xc3b2f61a72e188cfd44483fce1bc11d6a628766d.csv"

training_result_0x1e84 = pd.read_csv(path.join(path_to_folder,training_result_0x1e84_name),index_col=0)
training_result_0x78a = pd.read_csv(path.join(path_to_folder,training_result_0x78a_name),index_col=0)
training_result_0xc3b = pd.read_csv(path.join(path_to_folder,training_result_0xc3b_name),index_col=0)


In [4]:
path_history = "D:\AI_time_series_repos\TS_code\Forecasting\\basemodel\data\DF_three_series.csv"
fullYeardata = pd.read_csv(path_history,index_col= "date",parse_dates= True)
names = fullYeardata.columns
names

Index(['0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e',
       '0x1e84e4d5cf1f463147f3e4d566167597423d7769',
       '0xc3b2f61a72e188cfd44483fce1bc11d6a628766d'],
      dtype='object')

In [37]:
def get_ID_numbers(name:str, number, DF: pd.DataFrame):
    newSerie = DF.loc[[name],:].squeeze()
    newSerie = pd.to_numeric(newSerie)
    settings = pd.to_numeric(DF.columns[newSerie.values == number])
    return settings

def get_all_error_values(setting_ID,metric:str, DF: pd.DataFrame):
    collection = np.array([])
    for id in setting_ID:
        col = DF[str(id)+"_"+metric].values
        # print(col)
        collection = np.concatenate((collection,col), axis=None)
    return collection


In [38]:
def create_scatter_plot(name:str,numbers:list,metric:str,DF_log:pd.DataFrame,DF_error:pd.DataFrame, name_serie: str, colors:list = ['r','b','g']):
    unit = None
    if metric == "MAE" or metric == "RMSE":
        unit = "[kWh]"
    axis = figure_layout(titel= metric+" "+ name,ylabel="Error " + unit, xlabel="Number of run")
    for i in range(len(numbers)):
        number = numbers[i]
        color = colors[i]
        settings_try = get_ID_numbers(name,number,DF_log)
        out = get_all_error_values(settings_try,metric,DF_error)
        print(len(out),end="")

        axis.scatter(np.arange(1,len(out)+1),out,c=color,label=number)
    plt.legend(loc="upper right")
    out_name: str = name_serie+"_"+metric+"_"+ name+".png"
    # using output_path_pictures as global variable
    plt.savefig(path.join(output_path_pictures,out_name), dpi=300, facecolor='w', edgecolor='w', orientation='portrait', format=None,
                    transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)

def get_the_best_settings_MAE(DF_error: pd.DataFrame):
    """
    :param DF_error: dataFrame with all the errors collected during the parameter search
    :return: settings with the smallest MAE error
    """
    collection_errors = []
    for col_name in DF_error.columns:
        if col_name[-3:] == "MAE":
            collection_errors.append(DF_error[col_name].sum())
    found_min = min(collection_errors)
    found_id = collection_errors.index(found_min) + 1
    return found_id

def get_improvement(name:str,numbers:list,metric:str,DF_log:pd.DataFrame,DF_error:pd.DataFrame):
    collection = {}
    for i in range(len(numbers)):
        number = numbers[i]
        settings = get_ID_numbers(name,number,DF_log)
        out = get_all_error_values(settings,metric,DF_error)
        collection[str(number)] = np.mean(out)
    key_biggest_value = max(collection, key= collection.get)
    biggest_value = collection[key_biggest_value]
    del collection[key_biggest_value]
    collection = {key: (1 - collection[key]/biggest_value)*100 for key in collection.keys()}
    print("The improvement when choosing a certain value: %s"%collection)

    return collection

In [39]:
# create_scatter_plot("units_LSTM",[20,50],"MAE",logbook_0x78a,error_0x78a,'0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e')
# create_scatter_plot("layers_LSTM",[1,3],"MAE",logbook_0x78a,error_0x78a,'0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e')
# create_scatter_plot("lag_value",[48,96],"MAE",logbook_0x78a,error_0x78a,'0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e')
# create_scatter_plot("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0x78a,error_0x78a,'0x78a812ecd87a4b945e0d262aec41e0eb2b59fe1e')
#
#
# create_scatter_plot("units_LSTM",[20,50],"MAE",logbook_0x1e84,error_0x1e84,
# '0x1e84e4d5cf1f463147f3e4d566167597423d7769')
# create_scatter_plot("layers_LSTM",[1,3],"MAE",logbook_0x1e84,error_0x1e84,
# '0x1e84e4d5cf1f463147f3e4d566167597423d7769')
# create_scatter_plot("lag_value",[48,96],"MAE",logbook_0x1e84,error_0x1e84,
# '0x1e84e4d5cf1f463147f3e4d566167597423d7769')
# create_scatter_plot("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0x1e84,error_0x1e84,
# '0x1e84e4d5cf1f463147f3e4d566167597423d7769')
#
#
# create_scatter_plot("units_LSTM",[20,50],"MAE",logbook_0xc3b,error_0xc3b,'0xc3b2f61a72e188cfd44483fce1bc11d6a628766d')
# create_scatter_plot("layers_LSTM",[1,3],"MAE",logbook_0xc3b,error_0xc3b,'0xc3b2f61a72e188cfd44483fce1bc11d6a628766d')
# create_scatter_plot("lag_value",[48,96],"MAE",logbook_0xc3b,error_0xc3b,'0xc3b2f61a72e188cfd44483fce1bc11d6a628766d')
# create_scatter_plot("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0xc3b,error_0xc3b,'0xc3b2f61a72e188cfd44483fce1bc11d6a628766d')


In [40]:
found_id = get_the_best_settings_MAE(error_0x78a)
print(get_all_error_values(np.array([found_id]),"MAE", error_0x78a))
logbook_0x78a[str(found_id)]

[0.13670216 0.1392543  0.13636349]


units_LSTM                       20
layers_LSTM                       3
dropout_LSTM                      0
recurrent_dropout_LSTM            0
kernel_regularizer_LSTM        None
recurrent_regularizer_LSTM     None
bais_regularizer_LSTM          None
activity_regularizer_LSTM      None
units_DENSE                      50
layers_DENSE                      1
dropout_DENSE                     0
kernel_regularizer_DENSE       None
bais_regularizer_DENSE         None
activity_regularizer_DENSE     None
lag_value                        96
nb_epoch                          2
activation                     relu
batch_size_parameter             48
learning_rate                 0.001
patience                          0
shuffle                        True
repeat                            3
Name: 17, dtype: object

In [41]:
found_id = get_the_best_settings_MAE(error_0x1e84)
print(get_all_error_values(np.array([found_id]),"MAE", error_0x1e84))
logbook_0x1e84[str(found_id)]

[0.04256082 0.04304412 0.04239234]


units_LSTM                        50
layers_LSTM                        3
dropout_LSTM                       0
recurrent_dropout_LSTM             0
kernel_regularizer_LSTM         None
recurrent_regularizer_LSTM      None
bais_regularizer_LSTM           None
activity_regularizer_LSTM       None
units_DENSE                       50
layers_DENSE                       1
dropout_DENSE                      0
kernel_regularizer_DENSE        None
bais_regularizer_DENSE          None
activity_regularizer_DENSE      None
lag_value                         48
nb_epoch                           2
activation                      relu
batch_size_parameter              48
learning_rate                 0.0001
patience                           0
shuffle                         True
repeat                             3
Name: 10, dtype: object

In [42]:
found_id = get_the_best_settings_MAE(error_0xc3b)
print(get_all_error_values(np.array([found_id]),"MAE", error_0xc3b))
logbook_0xc3b[str(found_id)]

[0.09731915 0.10723963 0.09995547]


units_LSTM                       50
layers_LSTM                       3
dropout_LSTM                      0
recurrent_dropout_LSTM            0
kernel_regularizer_LSTM        None
recurrent_regularizer_LSTM     None
bais_regularizer_LSTM          None
activity_regularizer_LSTM      None
units_DENSE                      50
layers_DENSE                      1
dropout_DENSE                     0
kernel_regularizer_DENSE       None
bais_regularizer_DENSE         None
activity_regularizer_DENSE     None
lag_value                        96
nb_epoch                          2
activation                     relu
batch_size_parameter             48
learning_rate                 0.001
patience                          0
shuffle                        True
repeat                            3
Name: 23, dtype: object

In [43]:
get_improvement("units_LSTM",[20,50],"MAE",logbook_0x78a,error_0x78a)
get_improvement("layers_LSTM",[1,3],"MAE",logbook_0x78a,error_0x78a)
get_improvement("lag_value",[48,96],"MAE",logbook_0x78a,error_0x78a)
get_improvement("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0x78a,error_0x78a)
print("#"*100)
get_improvement("units_LSTM",[20,50],"MAE",logbook_0x1e84,error_0x1e84)
get_improvement("layers_LSTM",[1,3],"MAE",logbook_0x1e84,error_0x1e84)
get_improvement("lag_value",[48,96],"MAE",logbook_0x1e84,error_0x1e84)
get_improvement("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0x1e84,error_0x1e84)
print("#"*100)
get_improvement("units_LSTM",[20,50],"MAE",logbook_0xc3b,error_0xc3b)
get_improvement("layers_LSTM",[1,3],"MAE",logbook_0xc3b,error_0xc3b)
get_improvement("lag_value",[48,96],"MAE",logbook_0xc3b,error_0xc3b)
get_improvement("learning_rate",[0.0001,0.001,0.01],"MAE",logbook_0xc3b,error_0xc3b)



The improvement when choosing a certain value: {'20': 0.4460570445284695}
The improvement when choosing a certain value: {'3': 21.339499537871543}
The improvement when choosing a certain value: {'48': 7.5130668366641284}
The improvement when choosing a certain value: {'0.0001': 43.00387300698031, '0.001': 23.350514787014575}
####################################################################################################
The improvement when choosing a certain value: {'20': 0.062236708237239924}
The improvement when choosing a certain value: {'3': 10.603827641126596}
The improvement when choosing a certain value: {'48': 2.8444367336322607}
The improvement when choosing a certain value: {'0.0001': 12.773820034697392, '0.001': 2.513918858102304}
####################################################################################################
The improvement when choosing a certain value: {'50': 0.31899432463463606}
The improvement when choosing a certain value: {'3': 9.981884181677

{'0.0001': 17.939144985063336, '0.001': 13.210304891013024}